# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':', end=" ")
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
# Read in the data as Element Tree
document = ET.parse( './data/mondial_database.xml' )

In [6]:
# Find 10 countries with the lowest infant mortality rates
import numpy as np
import pandas as pd 

# Create a dictionary of countries with their infant mortality rates
country_dict = {}

for element in document.iterfind('country'):
    try:
        name = element.find('name').text
        infant_mortality = float(element.find('infant_mortality').text)
        country_dict[name] = [infant_mortality]
    except:
        pass

# Read data into a pandas dataframe and sort by infant mortality rates
country_df = pd.DataFrame.from_dict(country_dict, orient='index') 
country_df.reset_index(inplace=True)
country_df.columns = ['Country','Infant_Mortality']

country_df.sort_values('Infant_Mortality').head(10)

,Country,Infant_Mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [7]:
# Find 10 cities with the largest populations

# Create a dictionary of cities with their populations
city_dict = {}

for element in document.iter('country'):
    for subelement in element.iter('city'):
        try:
            name = subelement.find('name').text
            pop = int(subelement.find('population').text)
            city_dict[name] = pop
        except:
            pass

# Read data into a pandas dataframe and sort by population
city_df = pd.DataFrame.from_dict(city_dict, orient='index') 
city_df.reset_index(inplace=True)
city_df.columns = ['City','Population']

city_df.sort_values('Population', ascending=False).head(10)

,City,Population
1758,Seoul,10229262
1417,Mumbai,9925891
2550,São Paulo,9412894
1624,Jakarta,8259266
1247,Shanghai,8205598
1932,Ciudad de México,8092449
442,Moskva,8010954
1720,Tokyo,7843000
1246,Beijing,7362426
1463,Delhi,7206704


In [8]:
# Get a list of lists of all the elements to look in
element_list = [document.findall('country/province/city'),document.findall('country/city')]

# Flatten it into a single list to loop through below
element_list_all = [item for sublist in element_list for item in sublist]

data = {'City':[], 'Country':[], 'Year':[], 'Population':[]}

for city in element_list_all:
    country = city.attrib['country']
    name = ''
    population = 0
    year = 0
    
    for node in list(city):
        #print(node.tag)
        if node.tag == 'name':
            name = node.text
            #print (name + ":", end=" ")
        elif node.tag == 'population':
            # Note: if multiple population tags, the last is used
            # Population tags sorted by year, so last is most recent
            population = int(node.text)
            #print (population, end = " in year ")
            year = int(node.attrib['year'])
            #print (year)


    # only add cities with known populations
    if population != 0:
        data['Country'].append(country)
        data['City'].append(name)
        data['Population'].append(population)
        data['Year'].append(year)

# Put it in a dataframe and find the 10 largest cities by population
df = pd.DataFrame(data)
df.sort_values(by='Population', ascending=False).head(10)

,City,Country,Population,Year
1192,Shanghai,CN,22315474,2010
673,Istanbul,TR,13710512,2012
1353,Mumbai,IND,12442373,2011
421,Moscow,R,11979529,2013
1191,Beijing,CN,11716620,2010
2303,São Paulo,BR,11152344,2010
1193,Tianjin,CN,11090314,2010
915,Guangzhou,CN,11071424,2010
1399,Delhi,IND,11034555,2011
918,Shenzhen,CN,10358381,2010


In [9]:
# Find 10 ethnic groups with the largest overall populations 
# Sum of best/latest estimates over all countries

# Create a list of percentages of ethnic groups in various countries
ethnic_list = []

for element in document.iter('country'):
    country_name = element.find('name').text
    pop = int(element.find('population').text)
    for subelement in element.iter('ethnicgroup'):
        try:
            ethnic_name = subelement.text
            ethnic_pct = float(subelement.attrib['percentage'])
            ethnic_list.append([ethnic_name, country_name, ethnic_pct, pop])
        except:
            pass

# Read data into a pandas dataframe
ethnic_df = pd.DataFrame(ethnic_list) 
ethnic_df.columns = ['Ethnic_Group','Country','Ethnic_Percentage','Population']

In [10]:
# Creat a column of ethnic population for each ethnicity in each country
ethnic_df['Ethnic_Pop'] = ethnic_df.Ethnic_Percentage * ethnic_df.Population / 100
eth = ethnic_df[['Ethnic_Group','Ethnic_Pop']]
                
# Group by ethnicity and sum up each ethnicity's population in various countries
eth = eth.groupby('Ethnic_Group', as_index=False).sum()
eth.sort_values('Ethnic_Pop', ascending=False).head(10)

,Ethnic_Group,Ethnic_Pop
113,Han Chinese,4.975551e+08
89,European,1.928658e+08
120,Indo-Aryan,1.716454e+08
217,Russian,9.275844e+07
2,African,8.632937e+07
128,Japanese,8.170627e+07
105,German,6.623219e+07
77,Dravidian,5.959908e+07
84,English,4.231499e+07
176,Mestizo,3.554233e+07


In [11]:
# Find the name of the longest river and the countries it passes through

# Create a list of rivers with their length and the countries they pass through
river_list = []

for element in document.iter('river'):
    try:
        river_name = element.find('name').text
        #print(river_name + ':', end=' ')
        river_length = int(element.find('length').text)
        #print(river_length, end=' ')
        countries = element.attrib['country']
        #print(countries)
        river_list.append([river_name, river_length, countries])
    except:
        pass

# Read data into a pandas dataframe
river_df = pd.DataFrame(river_list) 
river_df.columns = ['River','Length','Countries']
river_df.sort_values('Length', ascending=False).head(10)

,River,Length,Countries
170,Amazonas,6448,CO BR PE
134,Jangtse,6380,CN
133,Hwangho,4845,CN
120,Lena,4400,R
197,Zaire,4374,RCB ZRE
135,Mekong,4350,CN LAO THA K VN
112,Irtysch,4248,R KAZ CN
182,Niger,4184,RMM RN WAN RG
156,Missouri,4130,USA
116,Jenissej,4092,R


In [12]:
# Find the name of the largest lake 

# Create a list of lakes with their sizea and the associated countries 
lake_list = []

for element in document.iter('lake'):
    try:
        lake_name = element.find('name').text
        lake_area = int(element.find('area').text)
        countries = element.attrib['country']
        lake_list.append([lake_name, lake_area, countries])
    except:
        pass

# Read data into a pandas dataframe
lake_df = pd.DataFrame(lake_list) 
lake_df.columns = ['Lake','Area','Countries']
lake_df.sort_values('Area', ascending=False).head(10)

,Lake,Area,Countries
44,Caspian Sea,386400,R AZ KAZ IR TM
96,Lake Superior,82103,CDN USA
68,Lake Victoria,68870,EAT EAK EAU
93,Lake Huron,59600,CDN USA
95,Lake Michigan,57800,USA
37,Dead Sea,41650,IL JOR WEST
70,Lake Tanganjika,32893,ZRE Z BI EAT
85,Great Bear Lake,31792,CDN
33,Ozero Baikal,31492,R
76,Lake Malawi,29600,MW MOC EAT


In [13]:
# Find the airport at the highest elevation

# Create a list of airports with their elevations 
airport_list = []

for element in document.iter('airport'):
    try:
        airport_name = element.find('name').text
        elevation = int(element.find('elevation').text)
        country = element.attrib['country']
        airport_list.append([airport_name, elevation, country])
    except:
        pass
    
# Read data into a pandas dataframe
airport_df = pd.DataFrame(airport_list) 
airport_df.columns = ['Airport','Elevation','Country']
airport_df.sort_values('Elevation', ascending=False).head(10)

,Airport,Elevation,Country
80,El Alto Intl,4063,BOL
212,Lhasa-Gonggar,4005,CN
230,Yushu Batang,3963,CN
787,Juliaca,3827,PE
789,Teniente Alejandro Velasco Astete Intl,3311,PE
82,Juana Azurduy De Padilla,2905,BOL
308,Mariscal Sucre Intl,2813,EC
779,Coronel Fap Alfredo Mendivil Duarte,2719,PE
781,Mayor General FAP Armando Revoredo Iglesias Ai...,2677,PE
666,Licenciado Adolfo Lopez Mateos Intl,2581,MEX
